## HINT FOR THE SOLUTION

By looking at the input, it is clear that the bunch of instructions can be split in 14 "subprograms" all starting storing an input digit in w. The blocks differ only fro 3 "parameters":

- the argument $d$ of the div instruction at the 5th instruction (it can only value 1 or 26)
- the value $q_1$ to be added to y in the 6th instruction
- the value $$q_2$ to be added to y in the 17th instruction of the block

by analyzing the input code, we can discover that the output z value of a subprogram only depends on w (the input digit) and the 3 aforementioned parameters. the rest of register values are only used as temporary storage and reset at the beginning of the next subprogram.

in particular, depending on the value of *d*, *w* and *q1* in the block, the output *z* will be:

1. when $d$ is 1:
    - if $(z\mod 26 + q_1)\neq w$ => $ outp = 26 z + w+ q_2$
    - if $(z\mod 26 + q_1)=w$ => $ outp = z$
2. when *d* is 26:
    - if $(z\mod 26 + q_1)\neq w$ => $ outp = 26(z \div 26) + w + q_2$
    - if $(z\mod 26 + q_1)==w$ => $ outp = z \div 26$

where $\div$ represents the integer division.
We can think of these expression as base 26 operations: 

- the first adds a new base 26 digit to the right of z ($w+q_2$ cannot be greater than 25)
- the second leaves z unchanged
- the third replaces the right digit with $w+q_2$
- the fourth operation removes one digit

Looking at the input code we also see that $w-q_1$ is always negative, thus for the first branch we always end in the first case. This means that each time the code enters in the first branch (it happens 7 times out of 14), a new digit is added to z. This also means that, if we want $z$ to be 0a at the end of the execution, the only way to obtain it is that for each of the remaining 7 subprograms (the ones with $d=26$) a character must be removed from $z$.
In conclusion: every subprogram taking branch 1. ends up in the first case (adding a new digit) and every subprogram goinginto branch 2 must end up in the second case (the left digit is removed from z)

This code implements also the alu logic, it has been used for testing the solution, but not for finding it. The only relevant method is ALU.find_solution, which returns both the highest and the lowest codes (respectively the answers to the first and second part).

In [1]:
from typing import Dict, Union, List, Optional
from copy import copy
import random
import cvxpy as cp
from collections import defaultdict

RegOrVal = Union[str, int]
Instruction = List[str]

class HashableDict(dict):
    def __hash__(self):
        return hash(frozenset(self.items()))

class ALU:
    def __init__(self, program :List[Instruction]) -> None:
        self._reset_register()
        self.subprograms = []
        subprogram = [program[0]]
        for instruction in program[1:]:
            if instruction[0] == "inp":
                self.subprograms.append(subprogram)
                subprogram = [instruction]
            else:
                subprogram.append(instruction)
        self.subprograms.append(subprogram)
    
    def get_params(self, subprogram_idx):
        div_z = int(self.subprograms[subprogram_idx][4][2])
        q1 = int(self.subprograms[subprogram_idx][5][2])
        q2 = int(self.subprograms[subprogram_idx][-3][2])
        return (div_z, q1, q2)
    
    def _reset_register(self)-> None:
        self.register : HashableDict[str, int] =  HashableDict(x=0, y=0, z=0, w=0)
        
    def _deref(self, arg: RegOrVal) -> int:
        if arg in self.register:
            return self.register[arg]
        else:
            return int(arg)

    def mul(self, arg1 :str, arg2 :RegOrVal)-> None:
        self.register[arg1] *= self._deref(arg2)

    def add(self, arg1 :str, arg2 :RegOrVal)-> None:
        self.register[arg1] += self._deref(arg2)
        
    def mod(self, arg1 :str, arg2 :RegOrVal)-> None:
        assert self._deref(arg1)>=0, f"called mod with arg1 ('{arg1}') {self._deref(arg1)}"
        assert self._deref(arg2)>0, f"called mod with arg1 ('{arg2}') {self._deref(arg2)}"
        self.register[arg1] = self.register[arg1] % self._deref(arg2)
        
    def div(self, arg1 :str, arg2 :RegOrVal)-> None:
        assert self._deref(arg2) != 0, f"called div with arg2 ('{arg2}') {self._deref(arg2)}"
        self.register[arg1] = self.register[arg1] // self._deref(arg2)
        
    def eql(self, arg1 :str, arg2 :RegOrVal)-> None:
        self.register[arg1] = 1 if self.register[arg1] == self._deref(arg2) else 0
        
    def inp(self, arg1 :str, val :int)-> None:
        self.register[arg1] = val
    
    def execute(self, program, input :List[int], initial_register = None, debug :bool = False):
        if initial_register is None:
                self._reset_register()
        else:
            self.register = copy(initial_register)
        idx = 0
        errors = False
        for i, inst in enumerate(program):
            if inst[0]=="inp":
                val = input[idx]
                getattr(self, inst[0])(inst[1], val)
                idx +=1
                if (debug):
                    print(f"{i}) instruction: {inst}, registry: {self.register}")
            else:
                try:
                    getattr(self, inst[0])(*inst[1:])
                    if (debug):
                        print(f"{i}) instruction: {inst}, registry: {self.register}")
                except AssertionError as e:
                    errors = True
                    print(f"error at instruction {i}: {e}")
                    break
        return (errors, self.register) 
    
    def find_solution(self) -> Optional[List[int]]:
        stack = []
        digits = [None] * 14
        res = {}
        for sp in range(len(self.subprograms)):
            div_z, q1, q2 = self.get_params(sp)
            if div_z==1:
                stack.append((sp,q2))
            else:
                matching_digit, q2 = stack.pop()
                diff = q2 + q1
                if diff < 0:
                    digits[matching_digit] = (-diff + 1, 9)
                    digits[sp] = (1, 9 + diff)
                else:
                    digits[matching_digit] = (1, 9-diff)
                    digits[sp] = (1+diff, 9)
        
        return ([d[0] for d in digits], [d[1] for d in digits])
        
                            
program = []
with open("input.txt", "r") as f:
    for line in f.read().splitlines():
        program.append(line.split())

alu = ALU(program)

res = alu.find_solution()
print("answer_1: " + ''.join(map(str, res[1])))
print("answer_2: " + ''.join(map(str, res[0])))
res = alu.execute(program, res[1])


answer_1: 92928914999991
answer_2: 91811211611981
